In [56]:
import pandas as pd
import time
import numpy as np

In [50]:
start_time = time.time()
whitelist_input_file_name = 'whitelist.xlsx'
whitelist = pd.read_excel(whitelist_input_file_name, header=0)
end_time = time.time()
print('load time:', end_time-start_time)

load time: 23.763453483581543


In [59]:
start_time = time.time()
covid_detection_input_file_name = 'covid_detection.xlsx'
covid_detection = pd.read_excel(covid_detection_input_file_name, header=0)
end_time = time.time()
print('load time:', end_time-start_time)

load time: 75.3331253528595


In [105]:
def clean_data(x):
    # check NULL value
    if pd.isnull(x):
        return '\\N'

    # remove newline
    s = str(x)
    if s.find('\n') != -1:
        print(s)
        s = s.replace('\n', '')
        print(s)

    # replace English comma with Chinese comma
    s = s.replace(',', '，')

    # if s end with '\', add a blank after s
    if s.endswith('\\'):
        s = s + ' '

    return s

In [28]:
print(whitelist['姓名'][2472])

TSHIBANDA. MUL IMUL OWAYI


In [37]:
whitelist.loc[34846]

街道                                    石岩街道
社区                                    浪心社区
网格                                    浪心31
所居住花园小区/城中村名称                         石岩新村
所属电子哨兵卡口名称                        石岩新村120栋
姓名                                     吴繁陆
性别                                       男
人员类型                                  内地居民
证件类型                               大陆居民身份证
证件号码                    440982198109235893
出生年月                   1981-09-23 00:00:00
手机号码                           13928441765
国籍                                      中国
是否暂离                                     否
户籍地址                    广东省化州市平定镇六赠良西冲村46号
工作单位所在市                                 深圳
工作单位所在行政区                              宝安区
工作单位名称                                 楼管员
工作单位地址                   广东省深圳市宝安区石岩街道浪心社区
是否纳入市网格办统计                               是
楼栋地址                              石岩新村120栋
楼栋编码                   4403060060021400130
房屋地址                           石岩新村120栋302
房屋编码       

In [54]:
print(covid_detection.columns)

Index(['姓名', '出生日期', '年龄', '电话号码', '机构所在地', '采样机构', '采样时间', '检测机构', '检测时间',
       '检测结果', '检测结果填报时间', '复核结果', '复核机构', '复核时间', '性别', '国家/地区', '居住地',
       '证件类型', '证件号码', '未提供有效证件原因', '检测人群分类', '应检尽检类别', '导入机构', '样本条形码',
       '样本类型', '检测项目', '采样点行政区划', '采样地点', '所在学校/单位名称', '备注1', '备注2', '采集类型',
       '导入时间', '创建人账号', '创建人姓名'],
      dtype='object')


In [106]:
start_time = time.time()
covid_detection_clean = covid_detection.apply(np.vectorize(clean_data))
end_time = time.time()
print('clean time:', end_time-start_time)
covid_detection_out_file_name = 'covid_detection.csv'
covid_detection_clean.to_csv(covid_detection_out_file_name, na_rep='NULL', header=True, index=False, encoding='utf-8')

clean time: 8.174000978469849


In [68]:
print(covid_detection['复核时间'][0])
print(type(covid_detection['复核时间'][0]))

nan
<class 'numpy.float64'>


In [102]:
print(np.isnan(covid_detection['复核时间'][0]))
print(pd.isnull(covid_detection['复核时间'][0]))

True
True


In [109]:
with open('load_covid_detection.sql', encoding='utf8') as f:
    print(f.read())
    # print(f.readline())
    # print(f.readlines())

# 1. drop temporary table storing new covid records
DROP TABLE langxin_community.new_covid_detection_records;

# 2. create a new temporary table storing new covid records
CREATE TABLE new_covid_detection_records (
    `id` INT UNSIGNED AUTO_INCREMENT,
    `姓名` VARCHAR(100),
    `出生日期` DATE,
    `年龄` INT,
    `电话号码` VARCHAR(20),
    `机构所在地` VARCHAR(100),
    `采样机构` VARCHAR(40),
    `采样时间` DATETIME,
    `检测机构` VARCHAR(40),
    `检测时间` DATETIME,
    `检测结果` VARCHAR(20),
    `检测结果填报时间` DATETIME,
    `复核结果` VARCHAR(20),
    `复核机构` VARCHAR(40),
    `复核时间` DATETIME DEFAULT NULL,
    `性别` VARCHAR(20),
    `国家/地区` VARCHAR(20),
    `居住地` VARCHAR(100),
    `证件类型` VARCHAR(20),
    `证件号码` VARCHAR(40),
    `未提供有效证件原因` VARCHAR(100),
    `检测人群分类` VARCHAR(40),
    `应检尽检类别` VARCHAR(40),
    `导入机构` VARCHAR(40),
    `样本条形码` VARCHAR(25),
    `样本类型` VARCHAR(20),
    `检测项目` VARCHAR(20),
    `采样点行政区划` VARCHAR(100),
    `采样地点` VARCHAR(100),
    `所在学校/单位名称` VARCHAR(40),
    `备注1` VARCHAR(100),
    `备注2` VARCHAR(1